In [3]:
# Carregar dados geográficos e previsões
# ============================================================
# 1️⃣ Carregar Bibliotecas, Modelo e Dados
# ============================================================
import pandas as pd
import joblib
import folium
import os

print("✅ Bibliotecas carregadas.")
# --- Carregar o pipeline treinado ---
# Este ficheiro contém a "fábrica" completa: pré-processador + modelo.
MODEL_PATH = "../data/model_inputs/tourism_model.pkl"
try:
    pipeline = joblib.load(MODEL_PATH)
    print(f"✅ Pipeline carregado de: {MODEL_PATH}")
except FileNotFoundError:
    print(f"🚨 ERRO: Pipeline não encontrado em '{MODEL_PATH}'.")
    print("Certifique-se de que executou o notebook de treino primeiro.")
    # Interromper a execução se o ficheiro não existir
    assert False, "Pipeline não encontrado."


# --- Carregar os dados originais ---
# Precisamos dos dados originais para obter as features e as coordenadas geográficas.
# ⚠️ ASSUNÇÃO: O CSV deve conter colunas com as coordenadas.
DATA_PATH = "../data/model_inputs/model_input.csv"
df = pd.read_csv(DATA_PATH)
df = df.dropna()
print(f"✅ Dados geográficos carregados: {df.shape[0]} localidades.")

# --- Fazer as previsões ---
# O pipeline espera receber um DataFrame com as mesmas colunas usadas no treino (exceto o alvo 'idh').
X_to_predict = df.drop(columns=['idh'], errors='ignore') # errors='ignore' para não dar erro se 'idh' não existir
predictions = pipeline.predict(X_to_predict)

# Adicionar as previsões ao DataFrame para fácil acesso
df['idh_predito'] = predictions

print("✅ Previsões realizadas com sucesso.")
display(df[['nome_ponto_turistico_clima', 'provincia_clima', 'idh', 'idh_predito']].head())



✅ Bibliotecas carregadas.
✅ Pipeline carregado de: ../data/model_inputs/tourism_model.pkl
✅ Dados geográficos carregados: 15 localidades.
✅ Previsões realizadas com sucesso.


,nome_ponto_turistico_clima,provincia_clima,idh,idh_predito
0,Quedas de Calandula,Malanje,0.543,0.60916
1,Miradouro da Lua,Luanda,0.590,0.62525
2,Museu Kulumbimbi,Zaire,0.726,0.67048
3,Reserva Parcial do Namibe,Namibe,0.624,0.60402
4,Fortaleza de São Miguel,Luanda,0.678,0.66271


In [4]:
# Criar mapa interativo com Folium
# ============================================================
# 2️⃣ Criar Mapa Interativo com Folium
# ============================================================

# Centralizar o mapa em Angola
map_center = [-11.2027, 17.8739]
m = folium.Map(location=map_center, zoom_start=6, tiles="CartoDB positron")

# Função para definir a cor do marcador com base no IDH predito
def get_marker_color(idh_predito):
    if idh_predito > 0.7:
        return 'darkgreen'  # Potencial Muito Alto
    elif idh_predito > 0.6:
        return 'green'      # Potencial Alto
    elif idh_predito > 0.5:
        return 'orange'     # Potencial Médio
    else:
        return 'red'        # Potencial Baixo

# Adicionar um marcador para cada ponto turístico no DataFrame
for idx, row in df.iterrows():
    # HTML para formatar o popup
    popup_html = f"""
    <h4>📍 {row['nome_ponto_turistico_clima']}</h4>
    <b>Província:</b> {row['provincia_clima']}<br>
    <hr>
    <b>IDH Real (nos dados):</b> {row['idh']:.3f}<br>
    <b>Potencial Predito (IDH):</b> <b>{row['idh_predito']:.3f}</b>
    """
    
    # Criar o popup
    popup = folium.Popup(popup_html, max_width=300)
    
    # Adicionar o marcador ao mapa
    # ⚠️ Verifique se os nomes 'lat_clima' e 'lon_clima' correspondem ao seu CSV
    folium.Marker(
        location=[row['lat_clima'], row['lon_clima']],
        popup=popup,
        tooltip=row['nome_ponto_turistico_clima'],
        icon=folium.Icon(color=get_marker_color(row['idh_predito']), icon='star')
    ).add_to(m)

print("✅ Mapa interativo criado na memória.")

✅ Mapa interativo criado na memória.


In [5]:
# Exibir mapa dentro do notebook
# ============================================================
# 3️⃣ Exibir Mapa Interativo
# ============================================================

# Ao executar esta célula, o mapa interativo será renderizado abaixo.
m

In [6]:
# Adicionar camadas auxiliares
# ============================================================
# 4️⃣ (Opcional) Adicionar Camadas Auxiliares ao Mapa
# ============================================================

# Adicionar um controle de camadas para ligar/desligar diferentes tiles
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain', attr='Stamen Terrain').add_to(m)
folium.LayerControl().add_to(m)

print("✅ Camadas auxiliares adicionadas ao mapa.")

# Re-exibir o mapa com as novas camadas (se a célula anterior já foi executada,
# o mapa original será atualizado automaticamente na próxima vez que for exibido).
# Para forçar a re-exibição imediata:
# m

✅ Camadas auxiliares adicionadas ao mapa.


In [7]:
# Entrega esperada: outputs/mapa_turismo.html
# ============================================================
# 5️⃣ Salvar o Mapa como um Ficheiro HTML
# ============================================================

# Definir o caminho e o nome do arquivo de saída
OUTPUT_DIR = "../outputs"
MAP_PATH = os.path.join(OUTPUT_DIR, "mapa_turismo.html")

# Criar o diretório se ele não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Salvar o objeto do mapa em um ficheiro HTML
m.save(MAP_PATH)

print(f"✅ Mapa salvo com sucesso em: {MAP_PATH}")

# Confirmação da entrega
if os.path.exists(MAP_PATH):
    print(f"\n✅ Entrega confirmada: o arquivo '{os.path.basename(MAP_PATH)}' existe no diretório '{OUTPUT_DIR}'.")
else:
    print(f"\n🚨 Atenção: o arquivo do mapa não foi encontrado no local esperado.")

✅ Mapa salvo com sucesso em: ../outputs\mapa_turismo.html

✅ Entrega confirmada: o arquivo 'mapa_turismo.html' existe no diretório '../outputs'.
